# <u>Knapsack Problem</u> 

# MAIN

In [135]:
import random
import time
from operator import itemgetter
from scipy.stats import beta
import numpy as np
from decimal import *

BRUTE_FORCE = 0
DYNAMIC_PROGRAMMING = 1

problem_solutions = []

# Get the number of item from terminal
nb_item = int(input("Enter number of item  - max :" ))

# Get the capacity of aliment from terminal
capacity = int(input('Enter the capacity of the sac : '))

## <span> 1. CLASS </span>

### <span style="color: yellow"> 1. Item </span>

In [136]:
class Item:
  def __init__(self, value, weight):
    self.value = value
    self.weight = weight
  
  def print(item):
    print("value = " + str(item.value) + " - weight = " + str(item.weight), end=' ')

  def print_solutions(items):
    for i in range(0, len(items)):
      Item.print(items[i])

#### <span style="color: GREEN"> 1. Total Weight</span>

In [137]:
def compute_total_weight(item):
    total_weight = 0
    
    for i in range(0, len(item)):
        total_weight += item[i].weight

    return total_weight

#### <span style="color: GREEN"> 2. Total Value</span>

In [138]:
def compute_total_value(item):
    total_value = 0
    
    for i in range(0, len(item)):
        total_value += item[i].value

    return total_value

### <span style="color: yellow"> 2. Solution </span>

In [139]:
class Solution:
  def __init__(self, time, name):
    self.name = name
    self.time = time
  
  def print(solution):
    print("name = " + solution.name + " - time = " + str(solution.time), end=' ')

## <span> 2. GENERATOR </span>

##### <span style="color: blue"> 1. Random generator</span>

In [140]:
def random_generator():
    choosed_items=[]

    # generate the aliments randomly
    for i in range (0, nb_item):

      weight = random.randint(1, capacity)
      value = random.randint(1, capacity)
      choosed_items.append(Item(value, weight))
        
    # return the choosed aliments with their weight
    return choosed_items


## <span> 3. ALGORITHM </span>

##### <span style="color: red"> 1. Brute Force</span>

In [141]:
import itertools

# calculate the total weight of the combination 
def compute_weight(combination_items):
  value, weight = 0, 0
  
  for item in combination_items:
    weight += item.weight
    value += item.value

  return value, weight 

# Brute force algorithm
def brute_force_solution(choosed_items):
  max_value = 0
  solution = ()

  for i in range(len(choosed_items) + 1):

    # create all possible solutions
    for combination_items in itertools.combinations(choosed_items, i):
      value, weight = compute_weight(combination_items) 

      # check the total weight of the combination 
      if value > max_value and weight <= capacity:
        solution = combination_items
        max_value = value

  return solution

##### <span style="color: red"> 2. Dynamic Programming </span>

In [142]:
def dynamic_programming_solution(choosed_items):
    table = [[0 for x in range(capacity + 1)] for x in range(nb_item + 1)] 
    for i in range(nb_item + 1): 
            for j in range(capacity + 1): 
                if i == 0 or j == 0: 
                    table[i][j] = 0
                elif choosed_items[i-1].weight <= j: 
                    table[i][j] = max(choosed_items[i-1].value  + table[i-1][j-choosed_items[i-1].weight],  table[i-1][j]) 
                else: 
                    table[i][j] = table[i-1][j] 
    return table[nb_item][capacity] 

## <span> 4. TEST  </span>

In [143]:
choosed_items = random_generator()

begin = time.time()
solution = brute_force_solution(choosed_items)
end = time.time()

problem_solutions.append(Solution((end - begin), BRUTE_FORCE))

begin = time.time()
s = dynamic_programming_solution(choosed_items)
end = time.time()

problem_solutions.append(Solution((end - begin), DYNAMIC_PROGRAMMING))

print("solution brute force = %s secondes" % problem_solutions[BRUTE_FORCE].time)
print("solution dynamic = %s secondes" % problem_solutions[DYNAMIC_PROGRAMMING].time)

print("capacity = " + str(capacity) + "\n")
print("choosed aliments = ")

# ! ********* For dev *******
Item.print_solutions(choosed_items)

print("========================\n")

total_weight = compute_total_weight(solution)
total_value = compute_total_value(solution)

for sol in (solution):
    Item.print(sol)
    print(";", end=' ')

print(s)

print()
print("- Total weight " + str(total_weight))
print("- Total value " + str(total_value))
    # ! **************

solution brute force = 3.1534790992736816 secondes
solution dynamic = 0.0 secondes
capacity = 10

choosed aliments = 
value = 6 - weight = 5 value = 8 - weight = 6 value = 7 - weight = 6 value = 5 - weight = 4 value = 4 - weight = 9 value = 3 - weight = 1 value = 10 - weight = 5 value = 6 - weight = 6 value = 9 - weight = 9 value = 9 - weight = 4 value = 2 - weight = 7 value = 5 - weight = 8 value = 7 - weight = 4 value = 10 - weight = 3 value = 8 - weight = 8 value = 2 - weight = 5 value = 6 - weight = 10 value = 3 - weight = 7 value = 9 - weight = 9 value = 2 - weight = 8 value = 10 - weight = 4 ========================

value = 3 - weight = 1 ; value = 10 - weight = 5 ; value = 10 - weight = 3 ; 23

- Total weight 9
- Total value 23


##### <span style="color: red"> 2. Branch and Bound </span>

In [144]:
class Node:
    def __init__(self, level, value, weight):
        self.items = []
        self.level = level
        self.value = value
        self.weight = weight

def sort_lists(values, weights, n):
    tmp = []
    for i in range(n):
        tmp.append((values[i], weights[i], values[i]/weights[i], i))

    tmp = sorted(tmp, key=itemgetter(2))
    tmp.reverse()
    sorted_values = []
    sorted_weights = []
    cor_id = []

    for k in tmp:
        sorted_values.append(k[0])
        sorted_weights.append(k[1])
        cor_id.append(k[3])

    return sorted_values, sorted_weights, cor_id
                        
    
    
def bound(node, n, values, weights):
    if node.weight >= capacity:
        return 0
    else:

        bound = node.value
        j = node.level + 1
        total_weight = node.weight

        while j <= n-1 and total_weight + weights[j] <= capacity:
            total_weight += + weights[j]
            bound += + values[j]
            j+=1

        k = j

        if k <= n-1:
            bound += (capacity - total_weight) * (values[k]/weights[k])
        return bound

def ks_branch_n_bound(values, weights, id):
    maxProfit = 0
    values, weights, id = sort_lists(values, weights, id)

    ks = []

    v = Node(-1, 0, 0)
    
    v.bound = bound(v, n, values, weights)

    ks.append(v)

    while ks:

        ks.sort(key=lambda i: i.bound)
        
        v = ks.pop(0)

        if v.bound > maxProfit: 

            node = Node(0, 0, 0)

            node.level = v.level + 1
            node.value = v.value + values[node.level]
            node.weight = v.weight + weights[node.level]

            node.items = v.items.copy()
            node.items.append(node.level)

            if node.weight <= capacity and node.value > maxProfit: 
                maxProfit = node.value
                #print(str(node.value))
                
            node.bound = bound(node, n, values, weights)

            if node.bound > maxProfit:
                ks.append(node)

            node_ = Node(node.level, v.value, v.weight)
            node_.bound = bound(node_, n, values, weights)
            node_.items = v.items.copy()

            if node_.bound > maxProfit:
                ks.append(node_)

    return maxProfit


n = len(objects)  # number of objects

weights = []      # list of weights
values = []       # list of values

for i in range(n):
  weights.append(int(objects[i][1]))  
  values.append(int(objects[i][2]))   

# Call function with counting time

begin = time.time()
res = ks_branch_n_bound(values, weights, n)
end = time.time()
result = '\nTime complexity: ' + str(end - begin) + '\nProfit: ' + str(res)

add_complexity('Branch and Bound', end ,begin)

print(result)



NameError: name 'objects' is not defined

##### <span style="color: red"> 3.1 Descending Order</span>


In [ ]:
def greedy_Algo_DESC(items,capacity):
    items = sorted(items,key=lambda item:item[1],reverse=True)
    print(items)
    chosen_item = {}
    profit = 0
    for i in range(len(items)):
        name,value,weight = items[i]
        num_of_fruit = (capacity - capacity% weight) / weight
        #capacity 9 weight 2 => (9-1)/4 = 4
        chosen_item[name] = num_of_fruit
        #we will take 'num_of_fruit' unity from the fruit with the name 'name'
        capacity = capacity % weight 
        profit += num_of_fruit*value
    return round(profit,2),chosen_item

In [ ]:

begin = time.time()
greedy_Algo_DESC(objects,capacity)
end = time.time()
result = '\nTime complexity: ' + str(end - begin) + '\nProfit: ' + str(res)

add_complexity('Descending Order', end ,begin)

print(result)


[('framboise', 5, 3), ('poire', 4, 4), ('Un classeur souple', 4, 4), ('RAPE', 4, 3), ('Un classeur rigide', 4, 3), ('RECHAUD', 4, 4), ('Un cahier', 3, 3), ('ananas', 3, 3), ('RECHAUD', 3, 3), ('framboise', 3, 2), ('Un petit cahier', 3, 3), ('lime', 3, 4), ('citron', 3, 3), ('raison', 2, 3), ('pomme', 2, 4), ('abricotpêche', 1, 4), ('RADIATEUR', 1, 4)]

Time complexity: 9.202957153320312e-05
Profit: 12


##### <span style="color: red"> 3.2 Ascending Order</span>

In [ ]:
def greedy_algo_ASC(items,capacity):
    items = sorted(items,key=lambda item:item[1])
    print(items)
    chosen_item = {}
    profit = 0
    for i in range(len(items)):
        name,value,weight = items[i]
        num_of_fruit = (capacity - capacity% weight) / weight
        #capacity 9 weight 2 => (9-1)/4 = 4
        chosen_item[name] = num_of_fruit
        #we will take 'num_of_fruit' unity from the fruit with the name 'name'
        capacity = capacity % weight 
        profit += num_of_fruit*value
    return round(profit,2),chosen_item

In [ ]:
begin = time.time()
greedy_algo_ASC(objects,capacity)
end = time.time()
result = '\nTime complexity: ' + str(end - begin) + '\nProfit: ' + str(res)

add_complexity('Ascending Order', end ,begin)

print(result)

[('abricotpêche', 1, 4), ('RADIATEUR', 1, 4), ('raison', 2, 3), ('pomme', 2, 4), ('Un cahier', 3, 3), ('ananas', 3, 3), ('RECHAUD', 3, 3), ('framboise', 3, 2), ('Un petit cahier', 3, 3), ('lime', 3, 4), ('citron', 3, 3), ('poire', 4, 4), ('Un classeur souple', 4, 4), ('RAPE', 4, 3), ('Un classeur rigide', 4, 3), ('RECHAUD', 4, 4), ('framboise', 5, 3)]

Time complexity: 0.0001850128173828125
Profit: 12


##### <span style="color: red"> 3.3 Value/weight Ratio </span>

In [ ]:
def greedy_Algo_W_by_V(items,capacity):
    items = sorted(items,key=lambda item:item[1]/item[2],reverse=True)
    chosen_items = {}
    profit = 0
    for i in range(len(items)):
        name,value,weight = items[i]
        num_of_fruit = (capacity - capacity% weight) / weight
        #capacity 9 weight 2 => (9-1)/4 = 4
        chosen_items[name] = num_of_fruit
        #we will take 'num_of_fruit' unity from the fruit with the name 'name'
        capacity = capacity % weight 
        profit += num_of_fruit*value
    return round(profit,2),chosen_items

In [ ]:
begin = time.time()
res = greedy_Algo_W_by_V(objects,capacity)
end = time.time()
result = '\nTime complexity: ' + str(end - begin) + '\nProfit: ' + str(res)

add_complexity('Value/weight Ratio', end ,begin)

print(result)


Time complexity: 5.412101745605469e-05
Profit: (8.0, {'framboise': 1.0, 'RAPE': 0.0, 'Un classeur rigide': 0.0, 'Un cahier': 0.0, 'poire': 0.0, 'ananas': 0.0, 'Un classeur souple': 0.0, 'RECHAUD': 0.0, 'Un petit cahier': 0.0, 'citron': 0.0, 'lime': 0.0, 'raison': 0.0, 'pomme': 0.0, 'abricotpêche': 0.0, 'RADIATEUR': 0.0})


##### <span style="color: red"> 3.4 0/1 knapsack problem </span>

In [ ]:
def dynamic_Programming(items,capacity):
    bag = [0 for i in range(capacity+1)]
    #list of (capacity+1 '0')
    for i in range(capacity+1):
        for j in range(len(items)):
            _,value,weight = items[j]
            if(weight < i):
                bag[i] = max(bag[i],bag[i-weight]+value) 
    return round(bag[capacity])

In [ ]:
dynamic_Programming(objects,capacity)

6

##### <span style="color: red"> 3.5 Fractional knapsack problem </span>

In [ ]:
def fractional_knapsack(items, capacity):
    
    items = sorted(items,key=lambda item:item[1]/item[2],reverse=True)
    
    max_value = 0
    fractions = [0]*len(items)
    for i in range(len(items)):
        if items[i][1] <= capacity:
            fractions[i] = 1
            max_value += items[i][2]
            capacity -= items[i][1]
        else:
            fractions[i] = capacity/items[i][1]
            max_value += items[i][2]*fractions[i]
            break
            
    return round(max_value), fractions

In [ ]:
begin = time.time()
res = fractional_knapsack(objects,capacity)
end = time.time()

result = '\nTime complexity: ' + str(end - begin) + '\nProfit: ' + str(res)

add_complexity('Branch and Bound', end ,begin)

result = '\nTime complexity: ' + str(end - begin) + '\nProfit: ' + str(res)

add_complexity('Value/weight Ratio', end ,begin)

print(result)



Time complexity: 4.792213439941406e-05
Profit: (3, [1, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


##### <span style="color: red"> 4. Dynamic Approach </span>

In [ ]:
def knapsack_dynamic_programming(items, capacity):
    weights = [item[1] for item in items]
    values = [item[2] for item in items]
    
    # Make the empty table 
    
    w,h = capacity + 1, len(values)
    table = [[0 for x in range(w)] for y in range(h)]

    for index in range(len(values)):
        for weight in range(w):
            # If the item weights more than the capacity at that column?
            # Take above value, that problem was solved
            if weights[index] > weight:
                table[index][weight] = table[index - 1][weight]
                continue
            # if the value of the item < capacity
            prior_value = table[index - 1][weight]
            #         val of current item  + val of remaining weight
            new_option_best = values[index] + table[index - 1][weight - weights[index]]
            table[index][weight] = max(prior_value, new_option_best)

    ##Get the max value        
            
    solution = max([x for y in table for x in y])
            
    return solution
    

begin = time.time()
res = knapsack_dynamic_programming(items[1],items[0])
end = time.time()
result = '\nTime complexity: ' + str(end - begin) + '\nProfit: ' + str(res)

add_complexity('Dynamic Approach', end ,begin)

print(result)


Time complexity: 8.320808410644531e-05
Profit: 12


In [ ]:
def knapsack_dynamic_programming_top_down(items, capacity):
    weights = [item[1] for item in items]
    values = [item[2] for item in items]    
    
    # Make the empty table 
    w,h = capacity + 1, len(values)
    table = [[0 for x in range(w)] for y in range(h)]
    
    return knapsack_recursive(table, values, weights, capacity, 0)

def knapsack_recursive(table, values, weights, capacity, currentIndex):

  if capacity <= 0 or currentIndex >= len(values):
    return 0

  if table[currentIndex][capacity] != 0:
    return table[currentIndex][capacity]

  value1 = 0
  if weights[currentIndex] <= capacity:
    value1 = values[currentIndex] + knapsack_recursive(
      table, values, weights, capacity - weights[currentIndex], currentIndex + 1)

  value2 = knapsack_recursive(
    table, values, weights, capacity, currentIndex + 1)

  table[currentIndex][capacity] = max(value1, value2)
  return table[currentIndex][capacity]

In [ ]:
begin = time.time()
res = knapsack_dynamic_programming_top_down(items[1], items[0])
end = time.time()
result = '\nTime complexity : ' + str(end - begin) + '\nProfit: ' + str(res)

add_complexity('Dynamic Approach Memoizing', end ,begin)

print(result)


Time complexity : 0.0001049041748046875
Profit: 12


In [ ]:
for item in gen_times:
    print(item)

('Brute Froce', Decimal('0.321441650390625'))
('Branch and Bound', Decimal('0.00043010711669921875'))
('Descending Order', Decimal('0.000092029571533203125'))
('Ascending Order', Decimal('0.0001850128173828125'))
('Value/weight Ratio', Decimal('0.0000541210174560546875'))
('Value/weight Ratio', Decimal('0.0000479221343994140625'))
('Dynamic Approach', Decimal('0.0000832080841064453125'))
('Dynamic Approach Memoizing', Decimal('0.0001049041748046875'))
